## 사용한 패키지 버전
pandas                        1.4.2
numpy                         1.21.5

In [1]:
import pandas as pd
import numpy as np
data1=pd.read_csv('data/loan_user_merge_first.csv')
data1=data1.drop(['Unnamed: 0'],axis=1)
data=data1.copy()

In [5]:
data.isnull().sum()

application_id                               0
loanapply_insert_time                        0
bank_id                                      0
product_id                                   0
loan_limit                                   0
loan_rate                                    0
is_applied                             3255482
user_id                                      0
birth_year                              128038
gender                                  128038
insert_time                                  0
credit_score                           1508618
yearly_income                                6
income_type                                  0
company_enter_month                     399201
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn             5885903
personal_rehabilitation_complete_yn    5885903
existing_loan

# 기대출수, 금액

In [9]:
# 기대출수 1일 때 평균값 확인 -->이상치가 많아서 중앙값이 좋을 듯!!! ㅠ
pd.options.display.float_format='{:.5f}'.format
data[data['existing_loan_cnt']==1]['existing_loan_amt'].describe()

# 기대출수 1일 때 금액이 0원인거 있는데 일단 테스트셋에서도 있어서 놔두기로 함

count      1639101.00000
mean      34238954.76850
std       56344476.33118
min              0.00000
25%        7000000.00000
50%       18000000.00000
75%       36000000.00000
max     1946000000.00000
Name: existing_loan_amt, dtype: float64

In [10]:
data[data['existing_loan_cnt']==1]['existing_loan_amt'].median()

18000000.0

In [11]:
# 기대출금액 null인 값 대체
only_amtnull=data[(data['existing_loan_cnt'].notnull())&(data['existing_loan_amt'].isnull())]['existing_loan_amt'].index
data.loc[only_amtnull,'existing_loan_amt']=round(18000000.0)

In [12]:

# 각 중앙값으로 대체- 이상치 때문에 평균보다는 중앙값이 좋겠다!
fill_mean_func = lambda g: g.fillna(g.median())
data['existing_loan_cnt']=data['existing_loan_cnt'].groupby([data['income_type'],data['employment_type'],data['houseown_type']]).apply(fill_mean_func)
data['existing_loan_amt']=data['existing_loan_amt'].groupby([data['income_type'],data['employment_type'],data['houseown_type']]).apply(fill_mean_func)

In [13]:
data.isnull().sum()

application_id                               0
loanapply_insert_time                        0
bank_id                                      0
product_id                                   0
loan_limit                                   0
loan_rate                                    0
is_applied                             3255482
user_id                                      0
birth_year                              128038
gender                                  128038
insert_time                                  0
credit_score                           1508618
yearly_income                                6
income_type                                  0
company_enter_month                     399201
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn             5885903
personal_rehabilitation_complete_yn    5885903
existing_loan

In [14]:
data[['existing_loan_cnt','existing_loan_amt']].describe()

,existing_loan_cnt,existing_loan_amt
count,13519858.00000,13519858.00000
mean,3.48401,69326152.39006
std,2.82528,86258195.25872
min,1.00000,0.00000
25%,2.00000,22000000.00000
50%,3.00000,43000000.00000
75%,4.00000,84000000.00000
max,229.00000,7512000000.00000


# credit_score

In [26]:
# credit_score
#employment type, income type별로 결측치를 채워주는 방식

pdf = pd.pivot_table(data,                # 피벗할 데이터프레임
                     index = 'employment_type',    # 행 위치에 들어갈 열
                     columns = 'income_type',    # 열 위치에 들어갈 열
                     values = ['credit_score'],     # 데이터로 사용할 열
                     aggfunc = ['median'])   # 중앙값 사용

#multi column 제거
pdf.columns = pdf.columns.get_level_values(2)


data['credit_score'] = data.apply(
    lambda row: pdf.loc[row['employment_type'], row['income_type']] if (pd.isnull(row['credit_score'])) and (pd.isnull(row['employment_type'])==False) and (pd.isnull(row['income_type'])==False) else row['credit_score'],
    axis=1)

In [27]:
data.isnull().sum()

application_id                               0
loanapply_insert_time                        0
bank_id                                      0
product_id                                   0
loan_limit                                   0
loan_rate                                    0
is_applied                             3255482
user_id                                      0
birth_year                              128038
gender                                       0
insert_time                                  0
credit_score                                 0
yearly_income                                6
income_type                                  0
company_enter_month                     399201
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn             5885903
personal_rehabilitation_complete_yn    5885903
existing_loan

# birth_year , gender

In [39]:
#birth_year 
# 유저 성별이 다른 신청서에는 있지만 결측치가 있는 유저들 처리
gender_null=set(data[(data['user_id'].isin(set(data[data['gender'].isnull()]['user_id']))) & (data['gender'].notnull())]['user_id'])

for i in gender_null:
    data.loc[(data['user_id']==i) & (data['birth_year'].isnull()), 'birth_year']=data[data['user_id']==i]['birth_year'].dropna().unique().item()
    data.loc[(data['user_id']==i) & (data['gender'].isnull()), 'gender']=data[data['user_id']==i]['gender'].dropna().unique().item()

# company_month 이상치 처리

In [40]:
#company_month 처리
# 이상치 처리
loan_user_new_copy = data.copy()

# company_enter_month 복사한 열 추가
loan_user_new_copy['company_enter_month_new'] = loan_user_new_copy['company_enter_month']
loan_user_new_copy['birth_year_new'] = loan_user_new_copy['birth_year']
### loan_user_new_1 : 'company_enter_month_new' 결측치가 없는 df
loan_user_new_1 = loan_user_new_copy[(loan_user_new_copy['birth_year_new'].notnull()) & (loan_user_new_copy['company_enter_month_new'].notnull())]

# 'birth_year_new', 'company_enter_month_new' 열 str으로 변환
loan_user_new_1['birth_year_new'] = loan_user_new_1['birth_year_new'].astype(str)
loan_user_new_1['company_enter_month_new'] = loan_user_new_1['company_enter_month_new'].astype(str)

# 'birth_year_new', 'company_enter_month_new' 열 연도만 남김(4자리)
loan_user_new_1['birth_year_new'] = loan_user_new_1['birth_year_new'].str[:4]
loan_user_new_1['company_enter_month_new'] = loan_user_new_1['company_enter_month_new'].str[:4]

# 'birth_year_new', 'company_enter_month_new' 열 정수형으로 변환
loan_user_new_1['company_enter_month_new'] = loan_user_new_1['company_enter_month_new'].astype(int)
loan_user_new_1['birth_year_new'] = loan_user_new_1['birth_year_new'].astype(int)

# <column 추가> 'diff_year' : 'birth_year_new', 'company_enter_month_new' 간의 차이를 계산한 column
loan_user_new_1['diff_year'] = loan_user_new_1['company_enter_month_new'] - loan_user_new_1['birth_year_new']

# a,b,c,d : 'employment_type'에 따라 df 분리
a = loan_user_new_1[loan_user_new_1['employment_type']=='정규직']
b = loan_user_new_1[loan_user_new_1['employment_type']=='계약직']
c = loan_user_new_1[loan_user_new_1['employment_type']=='기타']
d = loan_user_new_1[loan_user_new_1['employment_type']=='일용직']

# a에 대한 q1,q3,iqr 정의 및 a_outlier, a_inner 두 개로 a를 분리
a_q3 = a['company_enter_month_new'].quantile(0.75)
a_q1 = a['company_enter_month_new'].quantile(0.25)
a_iqr = a_q3 - a_q1
print(a_q1,a_q3,a_iqr)

a_outlier = a[(a['company_enter_month_new'] > (a_q3+1.5*a_iqr))|(a['company_enter_month_new'] < (a_q1-1.5*a_iqr))]
a_inner = a[(a['company_enter_month_new'] <= (a_q3+1.5*a_iqr)) & (a['company_enter_month_new'] >= (a_q1-1.5*a_iqr))]

# a_inner의 'diff_year' 평균값 도출
a_mod = a_inner['diff_year'].mode()[0]
a_mod

# a_outlier에 대해 'company_enter_month_new'는 'birth_year' + a_avg의 반올림값
a_outlier['company_enter_month_new'] = a_outlier['birth_year'] + round(a_mod)

# b에 대한 q1,q3,iqr 정의 및 b_outlier, b_inner 두 개로 b를 분리
b_q3 = b['company_enter_month_new'].quantile(0.75)
b_q1 = b['company_enter_month_new'].quantile(0.25)

b_iqr = b_q3 - b_q1
print(b_q1,b_q3,b_iqr)

b_outlier = b[(b['company_enter_month_new'] > (b_q3+1.5*b_iqr))|(b['company_enter_month_new'] < (b_q1-1.5*b_iqr))]
b_inner = b[(b['company_enter_month_new'] <= (b_q3+1.5*b_iqr)) & (b['company_enter_month_new'] >= (b_q1-1.5*b_iqr))]

# b_inner의 'diff_year' 평균값 도출
b_mod = b_inner['diff_year'].mode()[0]
b_mod

# b_outlier에 대해 'company_enter_month_new'는 'birth_year_new' + b_avg의 반올림값
b_outlier['company_enter_month_new'] = b_outlier['birth_year'] + round(b_mod)

# c에 대한 q1,q3,iqr 정의 및 c_outlier, c_inner 두 개로 c를 분리
c_q3 = c['company_enter_month_new'].quantile(0.75)
c_q1 = c['company_enter_month_new'].quantile(0.25)

c_iqr = c_q3 - c_q1
print(c_q1,c_q3,c_iqr)

c_outlier = c[(c['company_enter_month_new'] > (c_q3+1.5*c_iqr))|(c['company_enter_month_new'] < (c_q1-1.5*c_iqr))]
c_inner = c[(c['company_enter_month_new'] <= (c_q3+1.5*c_iqr)) & (c['company_enter_month_new'] >= (c_q1-1.5*c_iqr))]

# c_inner의 'diff_year' 평균값 도출
c_mod = c_inner['diff_year'].mode()[0]
c_mod

# c_outlier에 대해 'company_enter_month_new'는 'birth_year_new' + c_avg의 반올림값
c_outlier['company_enter_month_new'] = c_outlier['birth_year'] + round(c_mod)

# d에 대한 q1,q3,iqr 정의 및 d_outlier, d_inner 두 개로 d를 분리
d_q3 = d['company_enter_month_new'].quantile(0.75)
d_q1 = d['company_enter_month_new'].quantile(0.25)

d_iqr = d_q3 - d_q1
print(d_q1,d_q3,d_iqr)

d_outlier = d[(d['company_enter_month_new'] > (d_q3+1.5*d_iqr))|(d['company_enter_month_new'] < (d_q1-1.5*d_iqr))]
d_inner = d[(d['company_enter_month_new'] <= (d_q3+1.5*d_iqr)) & (d['company_enter_month_new'] >= (d_q1-1.5*d_iqr))]

# d_inner의 'diff_year' 평균값 도출
d_mod = d_inner['diff_year'].mode()[0]
d_mod

# d_outlier에 대해 'company_enter_month_new'는 'birth_year_new' + d_avg의 반올림값
d_outlier['company_enter_month_new'] = d_outlier['birth_year'] + round(d_mod)

result1=pd.concat([a_outlier,a_inner,b_outlier,b_inner,c_outlier,c_inner,d_outlier,d_inner])
result2 = loan_user_new_copy[(loan_user_new_copy['birth_year'].isnull()) & (loan_user_new_copy['company_enter_month_new'].isnull())]
result3 = loan_user_new_copy[(loan_user_new_copy['birth_year'].isnull()) & (loan_user_new_copy['company_enter_month_new'].notnull())]
result4 = loan_user_new_copy[(loan_user_new_copy['birth_year'].notnull()) & (loan_user_new_copy['company_enter_month_new'].isnull())]
data5=pd.concat([result1, result2, result3, result4]) # company 이상치 처리된 원본 데이터
data5=data5.sort_index()
data['company_enter_month']=data5['company_enter_month_new']

C:\Users\jus68\AppData\Local\Temp\ipykernel_9108\3994532240.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan_user_new_1['birth_year_new'] = loan_user_new_1['birth_year_new'].astype(str)
C:\Users\jus68\AppData\Local\Temp\ipykernel_9108\3994532240.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan_user_new_1['company_enter_month_new'] = loan_user_new_1['company_enter_month_new'].astype(str)
C:\Users\jus68\AppData\Local\Temp\ipykernel_9108\3994532240.py:16: SettingWithCopyWarning: 
A value is t

2016.0 2021.0 5.0


C:\Users\jus68\AppData\Local\Temp\ipykernel_9108\3994532240.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a_outlier['company_enter_month_new'] = a_outlier['birth_year'] + round(a_mod)


2020.0 2021.0 1.0


C:\Users\jus68\AppData\Local\Temp\ipykernel_9108\3994532240.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b_outlier['company_enter_month_new'] = b_outlier['birth_year'] + round(b_mod)
C:\Users\jus68\AppData\Local\Temp\ipykernel_9108\3994532240.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_outlier['company_enter_month_new'] = c_outlier['birth_year'] + round(c_mod)


2018.0 2021.0 3.0
2021.0 2022.0 1.0


C:\Users\jus68\AppData\Local\Temp\ipykernel_9108\3994532240.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_outlier['company_enter_month_new'] = d_outlier['birth_year'] + round(d_mod)


In [ ]:
# train, test set 분리하고 train에 있는 값 삭제
# 최빈값은 평균/중앙값과 마찬가지로 변수 간 상관관계를 고려하지 않는다. 데이터에 편향성을 만들어낼 수 있다. 따라서 train에 있는 값 삭제

ii=data[data.is_applied.isnull()].index
train=data.drop(ii)
test=data.loc[ii]
birth_null=train[(train['birth_year'].isnull())&(train['gender'].isnull())].index
loanuser=loanuser.drop(birth_null, axis=0)

In [62]:
# 남은 test에 있는 gender, birth_year 최빈값으로 채움
gender_freq = data['gender'].mode()[0]
data['gender'] = data['gender'].fillna(gender_freq)
data['birth_year'] = data['birth_year'].fillna(data['birth_year'].mode()[0])
data['gender'].isna().sum()

0

# company_month 결측치 처리

In [49]:
#### loan_user_new_2 : 'company_enter_month_new' 결측치가 있는 df
loan_user_new_2 = data[data['company_enter_month'].isnull()]

# 'birth_year_new' null값이 없어야 실행 가능
loan_user_new_2['birth_year_new'] = loan_user_new_2['birth_year'].astype(str)
loan_user_new_2['birth_year_new'] = loan_user_new_2['birth_year_new'].str[:4]
loan_user_new_2['birth_year_new'] = loan_user_new_2['birth_year_new'].astype(int)

# a2,b2,c2,d2 : 'employment_type'에 따라 df 분리
a2 = loan_user_new_2[loan_user_new_2['employment_type']=='정규직']
b2 = loan_user_new_2[loan_user_new_2['employment_type']=='계약직']
c2 = loan_user_new_2[loan_user_new_2['employment_type']=='기타']
d2 = loan_user_new_2[loan_user_new_2['employment_type']=='일용직']

# 'company_enter_month_new'에 각 카테고리별 diff_year의 avg 반올림값 더하기
a2['company_enter_month'] = a2['birth_year_new'] + round(a_mod)
b2['company_enter_month'] = b2['birth_year_new'] + round(b_mod)
c2['company_enter_month'] = c2['birth_year_new'] + round(c_mod)
d2['company_enter_month'] = d2['birth_year_new'] + round(d_mod)

C:\Users\jus68\AppData\Local\Temp\ipykernel_9108\1236342310.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan_user_new_2['birth_year_new'] = loan_user_new_2['birth_year'].astype(str)
C:\Users\jus68\AppData\Local\Temp\ipykernel_9108\1236342310.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan_user_new_2['birth_year_new'] = loan_user_new_2['birth_year_new'].str[:4]
C:\Users\jus68\AppData\Local\Temp\ipykernel_9108\1236342310.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [56]:
res=pd.concat([a2,b2,c2,d2],axis=0)
# 인덱스 정렬
res=res.sort_index()
# company null 대체
change_null=data[data['company_enter_month'].isnull()].index
data.loc[change_null]=res

In [67]:
data.isnull().sum()

application_id                               0
loanapply_insert_time                        0
bank_id                                      0
product_id                                   0
loan_limit                                   0
loan_rate                                    0
is_applied                             3255482
user_id                                      0
birth_year                                   0
insert_time                                  0
credit_score                                 0
yearly_income                                6
income_type                                  0
company_enter_month                          0
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn             5840712
personal_rehabilitation_complete_yn    5840712
existing_loan_cnt                            0
existing_loan

# 연소득

In [70]:
# 연소득
# 동일 id의 다른 application에서 yearly_income이 0으로 입력되어있는 것을 확인 가능
data[data['user_id']==670502.0]

# 따라서 'yearly_income'의 null 값의 경우 0으로 대체
data['yearly_income'] = data['yearly_income'].fillna(0)

In [69]:
data[data['user_id']==670502.0]['yearly_income']

4493087   0.00000
4493088   0.00000
4493089   0.00000
4493090   0.00000
4493091   0.00000
4493092   0.00000
4493093   0.00000
4493094       NaN
4493095   0.00000
4493096       NaN
4493097       NaN
4493098       NaN
4493099   0.00000
4493100   0.00000
4493101       NaN
4493102   0.00000
4493103       NaN
Name: yearly_income, dtype: float64

# 개인회생자
(A,B)라 했을 때
A = 개인회생자여부 column data
B = 개인회생자 납입 완료 여부 column data

0 - 개인회생자 아닌 사람(0,0)
1 - 개인회생자인데 납입중인사람(1,0)
2 - 개인회생자인데 납입완료된사람(1,1) = 일반인 돌아갈 예정인 사람
3 - 개인회생자였다가 일반인 된사람(0,1) 
-> 3번 이상치로 지우고 0,1,2 범주로 묶는다.


In [71]:
data.isnull().sum()

application_id                               0
loanapply_insert_time                        0
bank_id                                      0
product_id                                   0
loan_limit                                   0
loan_rate                                    0
is_applied                             3255482
user_id                                      0
birth_year                                   0
insert_time                                  0
credit_score                                 0
yearly_income                                0
income_type                                  0
company_enter_month                          0
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn             5840712
personal_rehabilitation_complete_yn    5840712
existing_loan_cnt                            0
existing_loan

In [88]:
# 이상치 제거
data=data.drop(data[(data['personal_rehabilitation_yn']==0)&(data['personal_rehabilitation_complete_yn']==1)].index,axis=0)

data['rehabilitation_yn']=3

# 0번
a_list=list(data[data['personal_rehabilitation_yn']==0].index)
data.loc[a_list,'rehabilitation_yn']=0
#1번
b_list=list(data[(data['personal_rehabilitation_yn']==1)&(data['personal_rehabilitation_complete_yn']==0)].index)
data.loc[b_list,'rehabilitation_yn']=1
#2번
c_list=list(data[(data['personal_rehabilitation_yn']==0)&(data['personal_rehabilitation_complete_yn']==1)].index)
data.loc[c_list,'rehabilitation_yn']=2

#3번- null


In [98]:
# 최종
data.isnull().sum()

application_id                               0
loanapply_insert_time                        0
bank_id                                      0
product_id                                   0
loan_limit                                   0
loan_rate                                    0
is_applied                             3255482
user_id                                      0
birth_year                                   0
insert_time                                  0
credit_score                                 0
yearly_income                                0
income_type                                  0
company_enter_month                          0
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
existing_loan_cnt                            0
existing_loan_amt                            0
gender                                       0
rehabilitatio

In [99]:
data.to_csv('loan_user_merge.csv')